In [1]:
import numpy as np
import random
import cv2
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Flatten
from keras.layers import Dropout
from keras.utils import to_categorical

!pip install scikit-plot
!pip install ucimlrepo

In [2]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
cdc_diabetes_health_indicators = fetch_ucirepo(id=891)

# data (as pandas dataframes)
X = cdc_diabetes_health_indicators.data.features
y = cdc_diabetes_health_indicators.data.targets

In [3]:
X_split = np.array_split(X, 4)
y_split = np.array_split(y, 4)

trainX_c, testX_c, trainY_c, testY_c = train_test_split(X_split[2], y_split[2], test_size=0.2)

In [4]:
model_c = Sequential()
model_c.add(Dense(3, activation='relu', input_shape=(trainX_c.shape[1], 1)))
model_c.add(Dropout(0.1))
model_c.add(Flatten())
model_c.add(Dense(1, activation='sigmoid'))
model_c.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_c.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 21, 3)             6         
                                                                 
 dropout (Dropout)           (None, 21, 3)             0         
                                                                 
 flatten (Flatten)           (None, 63)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 64        
                                                                 
Total params: 70 (280.00 Byte)
Trainable params: 70 (280.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history_c = model_c.fit(trainX_c, trainY_c, epochs=3)

Epoch 1/3
1586/1586 [==============================] - 16s 8ms/step - loss: 0.4104 - accuracy: 0.8505
Epoch 2/3
1502/1586 [===========================>..] - ETA: 0s - loss: 0.3513 - accuracy: 0.8615

In [ ]:
score_c = model_c.evaluate(testX_c, testY_c, verbose=0)
print(score_c)

In [ ]:
pesiTmp_c = model_c.get_weights()

In [ ]:
import requests as r
import json

In [ ]:
pesiTmp2_c = [pesiTmp_c[0][0], pesiTmp_c[1]]
# Conversione dei pesi per renderli compatibili con la codifica JSON e del server
pesi_c = '{\"l1\" : [[' + np.array2string(pesiTmp_c[0][0][0]) + '],[' + np.array2string(pesiTmp_c[0][0][1]) + '],[' + np.array2string(pesiTmp_c[0][0][2]) + ']], \"a\" : \"' + np.array2string(pesiTmp_c[1]) + '\"}'
pesi_json_c = json.loads(pesi_c)
print(pesi_c)

In [ ]:
url = 'http://93.65.162.11:3001'
x = r.post(url + '/nodo_c', data = pesi_c, headers={"Content-Type": "application/json"})
print(x)

In [ ]:
import json
media = r.get(url + '/pesi')
dict = json.loads(media.text)

pesi_tmp = np.array(dict['l1'])
pesi_tmp_c = np.array(dict['a'])

pesi_nodi = pesi_tmp.reshape((1,3))
pesi_archi = pesi_tmp_c.reshape((1,3))
pesi_archi[pesi_archi == None] = 0

In [ ]:
pesi_tmp_c = model_c.get_weights()

pesi_tmp_c[0][0][0] = pesi_nodi[0][0]
pesi_tmp_c[0][0][1] = pesi_nodi[0][1]
pesi_tmp_c[0][0][2] = pesi_nodi[0][2]

pesi_tmp_c[1][0] = pesi_archi[0][0]
pesi_tmp_c[1][1] = pesi_archi[0][1]
pesi_tmp_c[1][2] = pesi_archi[0][2]

In [ ]:
model_c.set_weights(pesi_tmp_c)
history_c = model_c.fit(trainX_c, trainY_c, epochs=3)
score_c = model_c.evaluate(testX_c, testY_c, verbose=0)